In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

csv_file_path = '/Users/jamellott23/Documents/GitHub/Unit-Test/vehicles_us.csv'

cars = pd.read_csv(csv_file_path)

print(cars)

       price  model_year           model  condition  cylinders fuel  odometer  \
0       9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1      25500         NaN      ford f-150       good        6.0  gas   88705.0   
2       5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3       1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4      14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   
...      ...         ...             ...        ...        ...  ...       ...   
51520   9249      2013.0   nissan maxima   like new        6.0  gas   88136.0   
51521   2700      2002.0     honda civic    salvage        4.0  gas  181500.0   
51522   3950      2009.0  hyundai sonata  excellent        4.0  gas  128000.0   
51523   7455      2013.0  toyota corolla       good        4.0  gas  139573.0   
51524   6300      2014.0   nissan altima       good        4.0  gas       NaN   

      transmission    type 

In [2]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [3]:
#Dropping all duplicates excluding date posted and days listened
cars = cars.drop_duplicates(subset=[col for col in cars.columns if col not in ['date_posted', 'days_listed']])

#Adding a manufacturer column by taking the first word from the model column.
cars['manufacturer'] = cars['model'].apply(lambda x: x.split()[0])

#Filling in the blanks for model_year and odometer
cars['model_year'] = cars['model_year'].fillna(0).astype(int)
cars['odometer'] = cars['odometer'].fillna(0).astype(int)

#Changing the model year and odometer columns from float to int dtype.
cars['model_year'] = cars['model_year'].astype(int)
cars['odometer'] = cars['odometer'].astype(int)

#Filling the 4wd column with 0's for cars that don't have 4wd.
cars['is_4wd'] = cars['is_4wd'].fillna(0).astype(int)

#Deleting the handful of cars made before 1950 or with odometers over 600000 or prices over $79000 as the data there is suspicious.
cars = cars[cars['model_year'] >= 1950]
cars = cars[cars['odometer'] <= 600000]
cars = cars[cars['price'] <= 79000]

#Converting date_posted to datetime and then creating a year_posted column
cars['date_posted'] = pd.to_datetime(cars['date_posted'])
cars['year_posted'] = cars['date_posted'].dt.year

#Creating a car age column
cars['age'] = cars['year_posted'] - cars['model_year']

#Listing all blank paint colors as unknown
cars['paint_color'] = cars['paint_color'].fillna("unknown")

#Changing all types to lowercase
cars['type'] = cars['type'].str.lower()

cars

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer,year_posted,age
0,9400,2011,bmw x5,good,6.0,gas,145000,automatic,suv,unknown,1,2018-06-23,19,bmw,2018,7
2,5500,2013,hyundai sonata,like new,4.0,gas,110000,automatic,sedan,red,0,2019-02-07,79,hyundai,2019,6
3,1500,2003,ford f-150,fair,8.0,gas,0,automatic,pickup,unknown,0,2019-03-22,9,ford,2019,16
4,14900,2017,chrysler 200,excellent,4.0,gas,80903,automatic,sedan,black,0,2019-04-02,28,chrysler,2019,2
5,14990,2014,chrysler 300,excellent,6.0,gas,57954,automatic,sedan,black,1,2018-06-20,15,chrysler,2018,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013,nissan maxima,like new,6.0,gas,88136,automatic,sedan,black,0,2018-10-03,37,nissan,2018,5
51521,2700,2002,honda civic,salvage,4.0,gas,181500,automatic,sedan,white,0,2018-11-14,22,honda,2018,16
51522,3950,2009,hyundai sonata,excellent,4.0,gas,128000,automatic,sedan,blue,0,2018-11-15,32,hyundai,2018,9
51523,7455,2013,toyota corolla,good,4.0,gas,139573,automatic,sedan,black,0,2018-07-02,71,toyota,2018,5


In [4]:
#Histogram of Car Prices with Color Coded Transmission Type

color_mapping = {'automatic': 'blue', 'manual': 'orange'}

transmission_prices = px.histogram(
    cars,
    x='price',
    color='transmission',
    color_discrete_map=color_mapping,
    labels={'price': 'Price', 'transmission': 'Transmission'},
    title='Overlapping Histogram of Car Prices by Transmission Type',
    nbins=20)

transmission_prices.show()

In [5]:
#Model and Year Prices

#Grouping the cars by model and year, then finding the average price for each car and how many there are of each
average_prices = cars.groupby(['model', 'model_year'])['price'].agg(['mean', 'count']).reset_index()

#Renaming the columns
average_prices.rename(columns={'mean': 'average_price', 'count': 'car_count'}, inplace=True)

#Changing the average price column to int dtype
average_prices['average_price'] = average_prices['average_price'].astype(int)

#Deleting any cars that don't have at least 5 ads listed
average_prices = average_prices[average_prices['car_count'] >= 5]

#Expensive Cars DF
expensive_cars_average = average_prices.sort_values(by='average_price', ascending=False).head(100)

expensive_cars_average

,model,model_year,average_price,car_count
62,cadillac escalade,2018,61212,6
66,chevrolet camaro,1969,54160,5
1847,ram 2500,2019,51719,10
61,cadillac escalade,2017,51054,9
175,chevrolet corvette,2018,50806,7
...,...,...,...,...
418,chevrolet suburban,2015,28410,23
1201,gmc sierra,2017,28244,10
774,ford f-250 sd,2014,28229,6
755,ford f-250,2016,28060,9


In [6]:
#A scatter plot of the most expensive 21st century cars listed for sale.

#Sorting for only 21st century cars
modern_expensive_cars_average = expensive_cars_average[expensive_cars_average['model_year'] > 1999]

#Allowing models to be color coded by making model category dtype
modern_expensive_cars_average.loc[:, 'model'] = modern_expensive_cars_average['model'].astype('category')

expensive_modern = px.scatter(
    modern_expensive_cars_average,
    x='model_year',
    y='average_price',
    color='model',
    labels={'model_year': 'Model Year', 'average_price': 'Average Price'},
    title='Scatter Plot: Most Expensive Modern Cars',
    hover_name='model',
    color_discrete_sequence=px.colors.qualitative.Set1,
)

# Show the plot
expensive_modern.show()


In [7]:
#Car Colors DF

#Grouping the cars by color and finding out how many there are of each and what the average price is.
car_colors = cars.groupby('paint_color').agg({'price': ['count', 'mean']}).reset_index()

#Renaming the columns
car_colors.columns = ['paint_color', 'car_count', 'average_price']

car_colors

,paint_color,car_count,average_price
0,black,5848,13029.183482
1,blue,3602,9886.219600
2,brown,951,9553.304942
3,custom,855,10070.505263
4,green,1106,7881.923146
5,grey,3985,11247.160100
6,orange,152,14770.460526
7,purple,80,6599.425000
8,red,3540,11036.275989
9,silver,5001,10393.909018


In [8]:
#Bar chart of average price by paint color.

color_mapping = {
    'blue': 'blue',
    'green': 'green',
    'black': 'black',
    'brown': 'saddlebrown',
    'grey': 'grey',
    'orange': 'orange',
    'purple': 'purple',
    'red': 'red',
    'silver': 'silver',
    'white': 'whitesmoke',
    'yellow': 'yellow',
    'unknown': 'lightskyblue',
    'custom': 'gold'}

#Changing the order to be more readable
paint_color_order = ['red', 'blue', 'green', 'yellow', 'orange', 'brown', 'purple', 'black', 'white', 'silver', 'grey', 'custom', 'unknown']


paint_colors = px.bar(car_colors, x='paint_color', y='average_price',
             color='paint_color', color_discrete_map=color_mapping,
             title='Average Price by Paint Color',
             labels={'paint_color': 'Paint Color', 'average_price': 'Average Price'},
             category_orders={'paint_color': paint_color_order})

paint_colors.show()


In [9]:
#4WD DF

#Getting the average price by car with and without 4wd.
is_4wd_counts = cars.groupby(['model', 'is_4wd']).agg({'price': ['count', 'mean']}).reset_index()

is_4wd_counts.columns = ['model', 'is_4wd', 'car_count', 'average_price']

with_4wd = is_4wd_counts[is_4wd_counts['is_4wd'] == 1].reset_index(drop=True)
without_4wd = is_4wd_counts[is_4wd_counts['is_4wd'] == 0].reset_index(drop=True)

#Merging the df's on the 'model' column
cars_4wd_final = pd.merge(with_4wd, without_4wd, on='model', suffixes=('_4wd', '_no_4wd'))

#Deleting any models that don't have at least 3 cars with 4wd and 10 cars total
cars_4wd_final = cars_4wd_final[(cars_4wd_final['car_count_4wd'] + cars_4wd_final['car_count_no_4wd']) >= 10]
cars_4wd_final = cars_4wd_final[cars_4wd_final['car_count_4wd'] >= 3]

#Calculating the difference in price between cars with 4wd and without
cars_4wd_final['price_difference'] = cars_4wd_final['average_price_4wd'] - cars_4wd_final['average_price_no_4wd']

print(cars_4wd_final)

print(cars_4wd_final['price_difference'].mean())

                model  is_4wd_4wd  car_count_4wd  average_price_4wd  \
0            acura tl           1             14        8793.785714   
1              bmw x5           1            199       10461.120603   
2       buick enclave           1             98       11527.591837   
3   cadillac escalade           1            215       17549.576744   
4    chevrolet camaro           1             24         621.791667   
..                ...         ...            ...                ...   
88      toyota sienna           1             40        9424.250000   
89      toyota tacoma           1            406       17618.399015   
90      toyota tundra           1            320       16284.246875   
91   volkswagen jetta           1              4        7872.500000   
92  volkswagen passat           1              8        4292.375000   

    is_4wd_no_4wd  car_count_no_4wd  average_price_no_4wd  price_difference  
0               0               186           6048.462366       2745.

In [10]:
#Shows the difference in price between each car model that had 4wd as an option. Average price differential is $2,775 dollars.

_4wd_difference = px.bar(cars_4wd_final, x='model', y='price_difference',
             color_discrete_map={'is_4wd_4wd': 'blue', 'is_4wd_no_4wd': 'orange'},
             title='Price Difference by Model (4WD vs. No 4WD)',
             labels={'price_difference': 'Price Difference', 'model': 'Car Model'},)

_4wd_difference.show()

In [14]:
#A scatterplot showing the price of cars versus their age, color coded by condition


#Changing the legend order to go from salvage to new
condition_order = ['salvage', 'fair', 'good', 'excellent', 'like new', 'new']

# Checkbox to show/hide conditions
show_conditions = st.checkbox("Select Conditions", value=True)

# Scatter plot
scatter_condition = px.scatter(cars, x='age', y='price', color='condition',
                               color_discrete_map={'excellent': 'blue', 'good': 'green', 'fair': 'orange', 'like new': 'purple', 'new': 'red', 'salvage': 'black'},
                               title='Scatter Plot of Car Prices by Age and Condition',
                               labels={'age': 'Age', 'price': 'Price', 'condition': 'Condition', 'model': 'Model'},
                               category_orders={'condition': condition_order},
                               hover_data={'model': True})

# Apply condition based on checkbox
if not show_conditions:
    scatter_condition.update_traces(visible='legendonly')

# Display the scatter plot
st.plotly_chart(scatter_condition)

DeltaGenerator()

In [12]:
# Creating a line chart looking at how each manufacturer maintains its value over time

#Grouping the cars by manufacturer and year and getting the average price and count for each
average_manuf_prices = cars.groupby(['manufacturer', 'model_year'])['price'].agg(['mean', 'count']).reset_index()

# Convert manufacturer names to the desired format
average_manuf_prices['manufacturer'] = average_manuf_prices['manufacturer'].apply(
    lambda x: 'GMC' if x == 'GMC' else x.title())

# Getting rid of cars before 1990 and over $40000 because the data isn't very interesting and gets into classics
average_manuf_prices_90 = average_manuf_prices[(average_manuf_prices['model_year'] >= 1990) & (average_manuf_prices['mean'] <= 40000)]

color_scale = px.colors.qualitative.Set1

man_mean_90 = px.scatter(average_manuf_prices_90, x='model_year', y='mean', color='manufacturer',
                 title='Average Price by Manufacturer and Model Year',
                 labels={'model_year': 'Model Year', 'mean': 'Average Price', 'manufacturer': 'Manufacturer'},
                 color_discrete_sequence=color_scale,
                 hover_data={'manufacturer': True, 'count': True, 'mean': True},
                 custom_data=['mean'],
                 trendline='lowess')

# Show the plot
man_mean_90.show()


In [13]:
#One more but set to 2010 to get a better look at newer car's pricing.

average_manuf_prices_10 = average_manuf_prices[average_manuf_prices['model_year'] >= 2010]

color_scale = px.colors.qualitative.Set1

man_mean_10 = px.scatter(average_manuf_prices_10, x='model_year', y='mean', color='manufacturer',
                 title='Average Price by Manufacturer and Model Year',
                 labels={'model_year': 'Model Year', 'mean': 'Average Price', 'manufacturer': 'Manufacturer'},
                 color_discrete_sequence=color_scale,
                 hover_data={'manufacturer': True, 'count': True, 'mean': True},
                 custom_data=['mean'],
                 trendline='lowess')

# Show the plot
man_mean_10.show()


